In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import time

import warnings
warnings.filterwarnings('ignore')

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs

np.random.seed(0)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


cpu
float64


In [2]:
train = pd.read_csv("data/train.csv")
Y = np.array(np.array(train["Cover_Type"].tolist()))
train = train.loc[:, train.columns != "Cover_Type"]
X = np.array(train)
feature_list = list(train.columns)
n = X.shape[0]

shuffle = np.random.permutation(np.arange(n))
X, Y = X[shuffle], Y[shuffle]

percent_in_dev = .2
dev_slice = int(percent_in_dev * n)

# Set some variables to hold test, dev, and training data.
dev_dat, dev_lab = X[:dev_slice], Y[:dev_slice]
train_dat, train_lab = X[dev_slice:], Y[dev_slice:]
# train_df = train.iloc[shuffle,:].iloc[dev_slice: , :]

n_dev = dev_dat.shape[0]
n_train = train_dat.shape[0]
n_feat = train_dat.shape[1]

def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data

train_lab_b = binarizeY(train_lab)
dev_lab_b = binarizeY(dev_lab)
n_class = train_lab_b[1].size

print(f'Dev size: {n_dev}')
print(f'Train size: {n_train}')
print(f'Features: {n_feat}')
print(f'Classes: {n_class}')
pd.DataFrame(train_dat).describe()

Dev size: 3024
Train size: 12096
Features: 55
Classes: 10


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
count,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,...,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000
mean,7527.118800,2751.081267,156.782821,16.483052,228.625165,51.281581,1707.330357,212.861690,218.987186,134.955936,...,0.022239,0.045883,0.040179,0.001571,0.006614,0.000661,0.001984,0.050761,0.043320,0.029266
std,4362.528179,416.896151,110.093786,8.443301,211.421953,61.433674,1319.372015,30.392477,22.776599,45.787111,...,0.147465,0.209240,0.196386,0.039603,0.081059,0.025710,0.044501,0.219517,0.203585,0.168558
min,1.000000,1863.000000,0.000000,0.000000,0.000000,-146.000000,0.000000,58.000000,99.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3734.500000,2378.000000,65.000000,10.000000,67.000000,5.000000,755.000000,196.000000,207.000000,106.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7513.500000,2753.500000,126.000000,15.000000,180.000000,33.000000,1310.000000,220.000000,223.000000,138.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,11283.250000,3105.000000,261.000000,22.000000,330.000000,79.000000,2263.250000,235.000000,235.000000,167.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,15119.000000,3849.000000,360.000000,50.000000,1318.000000,554.000000,6836.000000,254.000000,254.000000,248.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
## normalize training and dev
# pd.DataFrame(train_dat).head()

train_dat = normalize(train_dat, axis=0)
dev_dat = normalize(dev_dat, axis=0)
pd.DataFrame(train_dat).describe()




,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
count,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,...,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000
mean,0.007867,0.008990,0.007441,0.008093,0.006676,0.005827,0.007195,0.009001,0.009044,0.008610,...,0.001356,0.001948,0.001823,0.000360,0.000739,0.000234,0.000405,0.002049,0.001892,0.001555
std,0.004559,0.001362,0.005225,0.004145,0.006173,0.006980,0.005560,0.001285,0.000941,0.002921,...,0.008991,0.008882,0.008908,0.009086,0.009063,0.009090,0.009084,0.008859,0.008894,0.008959
min,0.000001,0.006088,0.000000,0.000000,0.000000,-0.016589,0.000000,0.002453,0.004088,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.003903,0.007771,0.003085,0.004910,0.001956,0.000568,0.003182,0.008288,0.008549,0.006763,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.007853,0.008998,0.005980,0.007364,0.005256,0.003750,0.005520,0.009303,0.009209,0.008805,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.011792,0.010146,0.012388,0.010801,0.009636,0.008976,0.009537,0.009937,0.009705,0.010655,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.015801,0.012577,0.017086,0.024548,0.038485,0.062947,0.028807,0.010741,0.010490,0.015823,...,0.060971,0.042448,0.045361,0.229416,0.111803,0.353553,0.204124,0.040357,0.043685,0.053149


In [4]:
## (1) Parameters
n_hidden_nodes = 50 
w_1 = theano.shared(np.asarray((np.random.randn(*(n_feat, n_hidden_nodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(n_hidden_nodes, n_hidden_nodes))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(n_hidden_nodes, n_hidden_nodes))*.01)))
w_4 = theano.shared(np.asarray((np.random.randn(*(n_hidden_nodes, n_class))*.01)))
params = [w_1, w_2, w_3]


## (2) Model
X = T.matrix()
Y = T.matrix()
# Two notes:
# First, feed forward is the composition of layers (dot product + activation function)
# Second, activation on the hidden layer still uses sigmoid
srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

def model(X, w_1, w_2, w_3, w_4, p=0):
    rectifier = lambda first_layer, second_layer : T.nnet.relu(T.dot(dropout(first_layer,p), second_layer))
    X_w_1 = rectifier(X, w_1)
    w_1_2 = rectifier(X_w_1, w_2)
    w_2_3 = rectifier(w_1_2, w_3)
    w_3_4 = rectifier(w_2_3, w_4)
    return T.nnet.softmax(w_3_4)
y_hat = model(X, w_1, w_2, w_3, w_4)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    acc = [0]
    for i in range(epochs):
        for start, end in zip(range(0, n_train, miniBatchSize), range(miniBatchSize, n_train, miniBatchSize)):
            cost = train(train_dat[start:end], train_lab_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        acc.append(np.mean(np.argmax(dev_lab_b, axis=1) == predict(dev_dat)))
        print(f'{i+1} accuracy = {acc[i+1]:.8f}')
    print(f'train time = {trainTime}')

gradientDescentStochastic(10)

start_time = time.time()
predict(dev_dat)   
print('predict time = %.2f' %(time.time() - start_time))

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '14060' (I am process '13408')
ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x,x}(TensorConstant{0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "C:\Users\sdste\AppData\Local\Programs\Python\Python310\lib\site-packages\theano\gof\opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "C:\Users\sdste\AppData\Local\Programs\Python\Python310\lib\site-packages\theano\tensor\opt.py", line 6515, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map,
  File "C:\Users\sdste\AppData\Local\Programs\Python\Python310\lib\site-packages\theano\gof\op.py", line 954, in make_thunk
    return self.make_c_thunk(node, storage_map, compute_map,
  File "C:\Users\sdste\AppData\Local\Programs\Python\Python310\lib\site-packages\theano\gof\op.py", line 

ImportError: DLL load failed while importing m885ff006a95d626dac547a7bdfdb471bbf058622ece2b4435e42316c4012ea56: The specified module could not be found.